# Testing a Model for Calculating Emissivity
In this notebook, we'll test out an emissivity model for calculating the power per unit volume for each loop in our AR using the expression,
$$
P(\lambda_{ij}) = 0.83\mathrm{Ab}(X)\left(\frac{N_j(X^{+m})}{N(X^{+m})}A_{ji}\Delta E_{ji}\right)\frac{N(X^{+m})}{N(X)}n_e
$$

The term in the parentheses is calculated by ChiantiPy and has units of erg s$^{-1}$. The fractional ionization $\frac{N(X^{+m})}{N(X)}$ can also be calculated by ChiantiPy assuming ionization equilbrium though there may be times when we will take non-equilibrium ionization into account and do this calculation ourselves.

The idea is that we'll have a base emission model class that does the basic CHIANTI calculations for emissivity and then also the fractional ionization calculation. To include a non-equilibrium ionization calculation, we can just subclass this base emission model and then include the NEI calculation for each loop.

In [2]:
import os
import subprocess
import pickle
import logging
import copy

import numpy as np
import scipy
import seaborn as sns
import ChiantiPy.core as ch
import ChiantiPy.tools as ch_tools
import h5py
import astropy.units as u
import astropy.constants as const
import sunpy.map
import sunpy.cm
from sunpy.net import vso
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors
from matplotlib.patches import Rectangle
from IPython.display import HTML

import synthesizAR
import synthesizAR.util
from synthesizAR.model_ext import EbtelInterface,PowerLawScaledWaitingTimes,UniformHeating
from synthesizAR.atomic import ChIon
from synthesizAR.instruments import InstrumentSDOAIA,InstrumentHinodeEIS

sns.set_context(context='notebook',font_scale=1.5)
logging.basicConfig(level=logging.INFO)
%matplotlib inline

/opt/anaconda3/envs/synthesizar/lib/python3.5/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


 using cli
 using CLI for selections
 reading chiantirc file


/opt/anaconda3/envs/synthesizar/lib/python3.5/site-packages/matplotlib/__init__.py:1357: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [ ]:
ar_root = os.path.join('/','data','datadrive2','ar_viz','emiss_model_testing')

## Field Setup

In [ ]:
client = vso.VSOClient()
result_hmi = client.query(
     vso.attrs.Time((2013, 1, 1, 7, 34, 0), (2013, 1, 1, 9, 0, 0)),
     vso.attrs.Instrument('HMI'),
     vso.attrs.Physobs('LOS_magnetic_field'),   # Physical observables
     vso.attrs.Sample(5000 * u.s)
)
data_hmi = client.get(result_hmi,methods=('URL-FILE_Rice','URL-FILE')).wait()

In [ ]:
crop = (u.Quantity([-140,90]*u.arcsec),u.Quantity([420,560]*u.arcsec))
resample = u.Quantity([100,100]*u.pixel)
field = synthesizAR.Skeleton(data_hmi[0],crop=crop,resample=resample)
zshape=50
zrange=u.Quantity([0.,150.]*u.arcsec)
field.extrapolate_field(zshape,zrange)

In [ ]:
field.extract_streamlines(200)

In [ ]:
field.peek(alpha=0.6)

## Loop Simulations

In [ ]:
field.make_loops()

In [ ]:
heating_options = {
    'duration':200.0,
    'duration_rise':100.0,
    'duration_decay':100.0,
    'average_waiting_time':5000.0,
    'stress_level':0.05,
    'alpha':-2.5,
    'delta_power_law_bounds':100,
    'waiting_time_scaling':1.0,
}
heating_model = UniformHeating(heating_options)

In [ ]:
ih = synthesizAR.util.InputHandler(os.path.join(os.environ['RESEARCH_DIR'],'ebtelPlusPlus/config/ebtel.example.cfg.xml'))
base_config = ih.lookup_vars()
base_config['c1_cond0'] = 6.0
base_config['use_adaptive_solver'] = True
base_config['tau'] = 1.0
base_config['adaptive_solver_error'] = 1e-12
base_config['adaptive_solver_safety'] = 0.01
base_config['total_time'] = 5000.0
ebtel_plug = EbtelInterface(base_config,heating_model)
field.configure_loop_simulations(ebtel_plug,
                                 parent_config_dir=os.path.join(ar_root,'hydro_config'),
                                 parent_results_dir=os.path.join(ar_root,'hydro_results'))

In [ ]:
for loop in field.loops:
    subprocess.call([os.path.join(os.environ['RESEARCH_DIR'],'ebtelPlusPlus/bin/ebtel++.run'),
                     '-c',loop.hydro_configuration['config_filename']])

In [ ]:
field.load_loop_simulations(ebtel_plug,savefile=os.path.join(ar_root,'loop_parameters.h5'))

In [ ]:
fig,ax = plt.subplots(2,1,figsize=(10,8),sharex=True)
for loop in field.loops:
    ax[0].plot(loop.time,loop.temperature[:,0],color=sns.color_palette('deep')[0],alpha=0.3)
    ax[1].plot(loop.time,loop.density[:,0],color=sns.color_palette('deep')[2],alpha=0.3)
ax[0].set_ylabel(r'$T$ ({0})'.format(loop.temperature.unit.to_string()))
ax[1].set_ylabel(r'$n$ ({0:latex})'.format(loop.density.unit))
ax[1].set_xlabel(r'$t$ ({})'.format(loop.time.unit))

In [ ]:
field.save_field(savedir=os.path.join(ar_root,'checkpoint'))

Reload the field here if needed.

In [ ]:
field = synthesizAR.Skeleton.restore_field(os.path.join(ar_root,'checkpoint'))

## Emissivity Model

First, construct a list of ions with the associated transitions. We'll start out with only four. Note that we can list multiple transitions per ion. These four lines are important when doing emission measure diagnostics on _Hinode_/EIS spectra.

| Ion | Wavelength ($\mathrm{\mathring{A}}$) |
|:---:|:------------------------------------:|
| Fe IX | 188.497 |
| Fe XII | 192.394 |
| Fe XII | 195.119 |
| Fe XVI | 262.976 |

In [ ]:
ions = [#{'name':'fe_9','wavelengths':[188.493]*u.angstrom},
        {'name':'fe_10','wavelengths':[184.537]*u.angstrom},
        {'name':'fe_12','wavelengths':[192.394,195.119]*u.angstrom},
        #{'name':'fe_13','wavelengths':[202.044]*u.angstrom}
        {'name':'fe_16','wavelengths':[262.976]*u.angstrom}
       ]

In [ ]:
class EmissivityModel(object):
    
    def __init__(self,ions,temperature=np.logspace(5,8,50)*u.K,
                 density=np.logspace(8,11,50)/(u.cm**3),energy_unit='erg',chianti_db_filename=None):
        self.density_mesh,self.temperature_mesh = np.meshgrid(density,temperature)
        self.wavelengths = np.array(sorted([w.value for ion in ions for w in ion['wavelengths']]))*ions[0]['wavelengths'].unit
        self.logger = logging.getLogger(name=type(self).__name__)
        
        if chianti_db_filename is None:
            chianti_db_filename = 'tmp_chianti_db.h5'
        self.logger.info('Creating CHIANTI HDF5 database in {}'.format(chianti_db_filename))
        self._build_chianti_db_h5(ions,chianti_db_filename)
        
        self.ions = []
        for ion in ions:
            self.logger.info('Creating ion {}'.format(ion['name']))
            tmp_ion = ChIon(ion['name'],np.ravel(self.temperature_mesh),np.ravel(self.density_mesh),chianti_db_filename)
            tmp_ion.meta['rcparams']['flux'] = energy_unit
            self.ions.append({'ion':tmp_ion,'transitions':ion['wavelengths']})
            
    def _build_chianti_db_h5(self,ions,filename):
        """
        Construct temporary HDF5 CHIANTI database
        """
        #create custom datatype for ragged scups arrays
        self._ragged_scups_dt = h5py.special_dtype(vlen=np.dtype('float64'))
        with h5py.File(filename,'a') as hf:
            for ion in ions:
                el = ion['name'].split('_')[0]
                ion_label = ion['name'].split('_')[-1]
                if os.path.join('/',el,ion_label) in hf:
                    continue
                self.logger.info('Building entry for {}'.format(ion['name']))
                #elvlc
                self.logger.info('Building elvlc entry for {}'.format(ion['name']))
                _tmp = ch_tools.io.elvlcRead(ion['name'])
                if _tmp['status']>0:
                    grp = hf.create_group(os.path.join('/',el,ion_label,'elvlc'))
                    self._check_keys(_tmp,grp)
                #wgfa
                self.logger.info('Building wgfa entry for {}'.format(ion['name']))
                try:
                    _tmp = ch_tools.io.wgfaRead(ion['name'])
                    grp = hf.create_group(os.path.join('/',el,ion_label,'wgfa'))
                    self._check_keys(_tmp,grp)
                except IOError:
                    pass
                #scups
                self.logger.info('Building scups entry for {}'.format(ion['name']))
                _tmp = ch_tools.io.scupsRead(ion['name'])
                if 'status' not in _tmp:
                    grp = hf.create_group(os.path.join('/',el,ion_label,'scups'))
                    self._check_keys(_tmp,grp)
                #psplups
                self.logger.info('Building psplups entry for {}'.format(ion['name']))
                _tmp = ch_tools.io.splupsRead(ion['name'],filetype='psplups')
                if 'file not found' not in _tmp:
                    grp = hf.create_group(os.path.join('/',el,ion_label,'psplups'))
                    self._check_keys(_tmp,grp)
            
    def _check_keys(self,chianti_dict,h5_group):
        """
        Checker for CHIANTI data dictionaries before reading into HDF5 file
        """
        for key in chianti_dict:
            self.logger.debug('Reading in key {}'.format(key))
            if key=='ref':
                h5_group.attrs[key] = '\n'.join(chianti_dict[key])
            elif type(chianti_dict[key]) is list or type(chianti_dict[key]) is type(np.array([])):
                data = np.array(chianti_dict[key])
                if '<U' in data.dtype.str:
                    data = data.astype('|S1')
                if data.dtype is np.dtype('O'):
                    ds = h5_group.create_dataset(key,(data.size,),dtype=self._ragged_scups_dt)
                    ds[:] = data
                else:
                    h5_group.create_dataset(key,data=data)
            else:
                h5_group.attrs[key] = chianti_dict[key]
            
    def _calculate_emissivity(self):
        """
        Calculate the level population times the A coefficent as a function of 
        density and temperature. Shape it into something more convenient.
        """
        for ion in self.ions:
            self.logger.info('Calculating emissivity for ion {}'.format(ion['ion'].meta['name']))
            wvl,emiss = ion['ion'].calculate_emissivity()
            transition_indices = [np.argwhere(wvl==t)[0][0] for t in ion['transitions']]
            ion['emissivity'] = [np.reshape(emiss[ti,:],self.temperature_mesh.shape) for ti in transition_indices]
    
    
    def _calculate_fractional_ionization(self):
        """
        Calculate fractional ionization as a function of temperature for each ion.
        Shape it into something more convenient
        """
        for ion in self.ions:
            ioneq = ion['ion'].calculate_ionization_equilibrium()
            ion['ionization_fraction'] = np.reshape(ioneq,self.temperature_mesh.shape)
            
            
    def calculate_emissivity(self,temperature,density):
        """
        Calculate emissivity for a given temperature and density
        """
        # interpolate indices
        nots_itemperature = scipy.interpolate.splrep(self.temperature_mesh[:,0].value,np.arange(self.temperature_mesh.shape[0]))
        nots_idensity = scipy.interpolate.splrep(self.density_mesh[0,:].value,np.arange(self.density_mesh.shape[1]))
        itemperature = scipy.interpolate.splev(np.ravel(temperature.value),nots_itemperature)
        idensity = scipy.interpolate.splev(np.ravel(density.value),nots_idensity)
        
        emiss = {}
        # calculate emissivity
        for ion in self.ions:
            self.logger.debug('Calculating emissivity for ion {}'.format(ion['ion'].meta['name']))
            if 'ionization_fraction' not in ion.keys():
                self._calculate_fractional_ionization()
            if 'emissivity' not in ion.keys():
                self._calculate_emissivity()
            for t,em in zip(ion['transitions'],ion['emissivity']):
                ion_key = '{} {} {}'.format(ion['ion'].meta['spectroscopic_name'],t.value,t.unit.to_string())
                self.logger.debug('Calculating emissivity for {}'.format(ion_key))
                _tmp_emiss = em*ion['ionization_fraction']
                _tmp = np.reshape(scipy.ndimage.map_coordinates(_tmp_emiss.value,
                                                                np.vstack([itemperature,idensity])),
                                  temperature.shape)
                _tmp = np.where(_tmp>0.0,_tmp,0.0)
                emiss[ion_key] = _tmp*em.unit*density*ion['ion'].abundance*0.83/(4*np.pi*u.steradian)
        
        return emiss
            

In [ ]:
em_model = EmissivityModel(ions,temperature=np.logspace(5,8,100)*u.K,density=np.logspace(8,11,50)/(u.cm**3),
                           chianti_db_filename=os.path.join(ar_root,'chianti_db.h5'))

In [ ]:
em_model._calculate_fractional_ionization()

In [ ]:
em_model.logger.setLevel(logging.INFO)
em_model._calculate_emissivity()

Now pass the emissivity model to the field object and let it calculate the emissivity at all desired wavelengths for all loops.

In [ ]:
field.calculate_emissivity(em_model,savefile=os.path.join(ar_root,'loop_emissivity.h5'))

In [ ]:
fig,axes = plt.subplots(4,1,figsize=(8,8),sharex=True,sharey=True)
for loop in field.loops:
    axes[0].plot(loop.time,loop.get_emissivity(184.537*u.angstrom)[:,0],alpha=0.2,c='b')
    axes[0].set_title(r'Fe X 184.537 Angstrom')
    axes[1].plot(loop.time,loop.get_emissivity(192.394*u.angstrom)[:,0],alpha=0.2,c='b')
    axes[1].set_title(r'Fe XII 192.394 Angstrom')
    axes[2].plot(loop.time,loop.get_emissivity(195.119*u.angstrom)[:,0],alpha=0.2,c='b')
    axes[2].set_title(r'Fe XII 195.119 Angstrom')
    axes[3].plot(loop.time,loop.get_emissivity(262.976*u.angstrom)[:,0],alpha=0.2,c='b')
    axes[3].set_title(r'Fe XVI 262.976 Angstrom')
axes[0].set_yscale('log')
axes[0].set_ylim([10,1e+5])
axes[1].set_ylabel(r'$\epsilon$ ({})'.format(loop.get_emissivity(262.976*u.angstrom).unit))
axes[-1].set_xlabel(r'$t$ ({})'.format(loop.time.unit))

## Binning Counts for Detector.
Now we want to use our *Hinode*/EIS detector to detect our synthesized emission and bin it appropriately.

In [ ]:
eis = InstrumentHinodeEIS('/home/wtb2/Documents/Forward_Model/instruments/Hinode_EIS/',[0,5000]*u.s)

We've read in all of the possible channels, but what if we don't want to calculate the counts for every channel? Let's remove the channels we don't want. I guess we could also do this with a flag somewhere? But this means checking for a flag each time we loop through the instruments for each channel and that seems clumsy too.

We only want to keep the channels that are relevant to the emission we've calculated.

In [ ]:
eis.channels = [eis.channels[3],eis.channels[10],eis.channels[16],eis.channels[31]]

Now declare the observer object.

In [ ]:
observer = synthesizAR.Observer(field,[eis],ds=0.3*u.arcsec)

Build the detector files.

In [ ]:
observer.build_detector_files(ar_root)

Now, calculate the detector counts for each channel.

In [ ]:
observer.calculate_detector_counts()

Finally, bin the counts and save them as FITS files.

In [ ]:
observer.bin_detector_counts(os.path.join(ar_root,'maps'),apply_psf=False)

## Visualization

In [ ]:
fits_file_template = os.path.join(ar_root,'maps','Hinode_EIS','FeXVI 262.976','map_t{time:06d}.fits')

In [ ]:
fig = plt.figure(figsize=(8,8))
tmp_map = sunpy.map.Map(fits_file_template.format(time=150))#.submap([-80,30]*u.arcsec,[430,540]*u.arcsec)
#ax = fig.add_subplot(1,1,1,projection=tmp_map)
ax = fig.gca(projection=tmp_map)
tmp_map.plot(axes=ax,cmap=sunpy.cm.get_cmap('hinodexrt'),
             vmin=0.01,vmax=100)
ax.add_patch(Rectangle(((-70,490)*u.arcsec).to(u.deg).value,
                       (100*u.arcsec).to(u.deg).value,
                       (1.5*u.arcsec).to(u.deg).value,
                      transform=ax.get_transform('world'),lw=1,edgecolor='white',fill=False))
#tmp_map.draw_rectangle((-100, 490)*u.arcsec, 150*u.arcsec, 1*u.arcsec,lw=1)

For a particular channel, let's loop over each timestep and take a slice out of the map. Then, we can plot this slice as a function of time.

In [ ]:
x1,y1 = tmp_map.data_to_pixel(-100*u.arcsec,490*u.arcsec)
x2,y2 = tmp_map.data_to_pixel(50*u.arcsec,490*u.arcsec)

In [ ]:
for i in range(len(eis.observing_time)):
    tmp_map = sunpy.map.Map(fits_file_template.format(time=i))
    tmp_slice = tmp_map.data[int(np.ceil(y1.value)):int(np.ceil(y2.value))+1,
                             int(np.ceil(x1.value)):int(np.ceil(x2.value))+1]
    if i == 0:
        stacked_slice = tmp_slice
    else:
        stacked_slice = np.vstack([stacked_slice,tmp_slice])

In [ ]:
xmesh,tmesh = np.meshgrid(np.arange(x1.value,x2.value+1,1),
                          eis.observing_time.value)
fig = plt.figure(figsize=(8,10))
ax = fig.gca()
pcm = ax.pcolormesh(xmesh,tmesh,stacked_slice,cmap=sunpy.cm.get_cmap('hinodexrt'),
                    #norm = matplotlib.colors.LogNorm(vmin=1,vmax=500)
                    )
cbar = fig.colorbar(pcm,ax=ax,)
ax.set_xlim([x1.value,x2.value])
ax.set_ylim([eis.observing_time[0].value,eis.observing_time[-1].value])
ax.invert_yaxis()
ax.set_ylabel(r'$t$ ({0:latex})'.format(eis.observing_time.unit))
ax.set_xlabel(r'$x$ ({0:latex})'.format(tmp_map.xrange.unit))
cbar.set_label(r'$I$ ({0:latex})'.format(u.Unit(tmp_map.meta['bunit'])))

Now, let's try to make a movie using these four channels.

In [ ]:
intensity_ranges = {'FeX 184.537':[1,100],
                    'FeXII 192.394':[10,500],
                    'FeXII 195.119':[100,1000],
                    'FeXVI 262.976':[10,100]}

In [ ]:
fits_file_template = os.path.join(ar_root,'maps','Hinode_EIS','{channel}','map_t{time:06d}.fits')
save_file_template = os.path.join(ar_root,'maps','Hinode_EIS','figs','all_channels_t{time:06d}.pdf')
fig = plt.figure(figsize=(10,6))
for j,time in enumerate(eis.observing_time):
    fig.suptitle(r'$t=${0} ({1})'.format(time,eis.observing_time.unit),fontsize=16)
    for i,channel in enumerate(eis.channels):
        _tmp_map = sunpy.map.Map(fits_file_template.format(channel=channel['name'],time=j))
        ax = fig.add_subplot(2,2,i+1,projection=_tmp_map)
        _tmp_map.plot(axes=ax,vmin=intensity_ranges[channel['name']][0],
                      vmax=intensity_ranges[channel['name']][1],
                      annotate=False,cmap=sunpy.cm.get_cmap('hinodexrt'))
        ax.set_title(r'{0} $\mathrm{{\mathring{{A}}}}$'.format(channel['name']))
        if i>1:
            ax.set_xlabel(r'$x$ ({})'.format(_tmp_map.xrange.unit))
        if i==0 or i==2:
            ax.set_ylabel(r'$y$ ({})'.format(_tmp_map.yrange.unit))
    plt.savefig(save_file_template.format(time=j))
    fig.clf()

In [ ]:
%%bash
convert -delay 3 -loop 0 /data/datadrive2/ar_viz/emiss_model_testing/maps/Hinode_EIS/figs/*.pdf eis_all_channels.gif

In [ ]:
%%HTML
<img src="eis_all_channels.gif" />